In [1]:
# ================================================================
# 💰 ML_Customer_Spending_Prediction.ipynb
# Predict total spending per customer using Fact_Orders + Dim_Customers
# Author: Muhammad Saad
# ================================================================

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as _sum, count, avg, col, current_timestamp
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score


StatementMeta(, 260ff250-e021-44d4-bd15-ac8b8a201bac, 3, Finished, Available, Finished)

In [2]:
# 2️⃣ Load data from Gold layer
fact_orders = spark.read.table("GoldLakehouse.dbo.Fact_Orders")
dim_customers = spark.read.table("GoldLakehouse.dbo.Dim_Customers")

print(f"Loaded {fact_orders.count()} orders and {dim_customers.count()} customers")


StatementMeta(, 260ff250-e021-44d4-bd15-ac8b8a201bac, 4, Finished, Available, Finished)

Loaded 1000 orders and 1000 customers


In [3]:

# 3️⃣ Aggregate spending data from orders
customer_spend_df = (
    fact_orders
    .groupBy("Customer_ID")
    .agg(
        _sum("Order_Total").alias("Total_Spent"),
        count("Order_ID").alias("Total_Orders"),
        avg("Price").alias("Avg_Price")
    )
)

# 4️⃣ Join with customer demographics
df_joined = (
    customer_spend_df.alias("sp")
    .join(dim_customers.alias("cu"), col("sp.Customer_ID") == col("cu.Customer_ID"), "left")
    .select(
        col("sp.Customer_ID"),
        col("Total_Spent"),
        col("Total_Orders"),
        col("Avg_Price"),
        col("cu.Age"),
        col("cu.Gender")
    )
)

# 5️⃣ Convert to Pandas for model training
pdf = df_joined.toPandas()

# Encode gender (M/F → numeric)
pdf["Gender"] = pdf["Gender"].fillna("Unknown")
pdf["Gender"] = pdf["Gender"].map({"male": 1, "female": 0}).fillna(0)

# 6️⃣ Prepare features and labels
X = pdf[["Total_Orders", "Avg_Price", "Age", "Gender"]]
y = pdf["Total_Spent"]

# Handle missing values
X = X.fillna(0)
y = y.fillna(0)

# 7️⃣ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 8️⃣ Train Random Forest model
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 9️⃣ Evaluate model
preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

print("✅ Model trained successfully")
print(f"📊 MAE: {mae:.2f} | R²: {r2:.3f}")


StatementMeta(, 260ff250-e021-44d4-bd15-ac8b8a201bac, 5, Finished, Available, Finished)

✅ Model trained successfully
📊 MAE: 39770.70 | R²: 0.692


In [5]:

# 🔟 Predict total spending for all customers
pdf["Predicted_Spent"] = model.predict(X)

# 1️⃣1️⃣ Convert predictions back to Spark DataFrame
predictions_sdf = spark.createDataFrame(pdf[["Customer_ID", "Total_Spent", "Predicted_Spent", "Total_Orders", "Avg_Price", "Age", "Gender"]])

predictions_sdf.write.format("delta").mode("overwrite").saveAsTable("Gold_Customer_Spending_Predictions")

print("✅ Gold_Customer_Spending_Predictions table created successfully")



StatementMeta(, 260ff250-e021-44d4-bd15-ac8b8a201bac, 7, Finished, Available, Finished)

✅ Gold_Customer_Spending_Predictions table created successfully
